## IE: 692  - Advanced Process Mining
### Replication Study: Prescriptive Process Monitoring for Cost-Aware Cycle Time Reduction
#### Group 8 - Deidamea Bajri, Flavjo Tomori, Sara Koni, Stiliana Jano, Tomas Muça

#### This Jupyter Notebook contains the preprocessing steps of BPI 2018 for applying the original model.

Importing all required packages.

In [1]:
import pm4py
import pandas as pd
import numpy as np
import re
from datetime import datetime
from tqdm import tqdm
import warnings
from nltk.tokenize import word_tokenize
import calendar
import random

warnings.filterwarnings("ignore")

Importing the log, filtering it with start and end activities, and converting it to a dataframe.

In [2]:
if __name__ == "__main__":
    
    #importing the log
    log = pm4py.read_xes(r'C:\Users\user\Downloads\BPI Challenge 2018.xes')
    #filter log based on start and end activities
    start_activity = "mail income"
    end_activity = "finish payment"
    filtered_log = pm4py.filter_start_activities(log, start_activity)
    filtered_log = pm4py.filter_end_activities(filtered_log, end_activity)
    #converting the log into a dataframe
    original_dataframe = pm4py.convert_to_dataframe(filtered_log)

parsing log, completed traces ::   0%|          | 0/43809 [00:00<?, ?it/s]

In [4]:
original_dataframe.head()

,success,org:resource,docid_uuid,doctype,subprocess,docid,activity,note,eventid,identity:id,...,case:concept:name,case:penalty_amount1,case:payment_actual1,case:amount_applied1,case:penalty_amount2,case:payment_actual2,case:amount_applied2,case:penalty_amount3,case:payment_actual3,case:amount_applied3
0,True,0;n/a,CD3DC291-76C6-420A-B3F1-7C808970915B,Payment application,Application,-18008611495569447,mail income,none,null,510B5333-731A-40FD-B7D6-FC149E50E961,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,0;n/a,CD3DC291-76C6-420A-B3F1-7C808970915B,Payment application,Application,-18008611495569447,mail valid,none,null,F1DD45EF-80BF-46A5-97D6-CC5886DD2D23,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,0;n/a,7CB69360-6D10-426F-A426-DDE3E24E4334,Entitlement application,Main,-18008615298673397,mail valid,none,null,AA02CA32-D021-4264-A7CB-660A9D603EFC,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,0;n/a,7CB69360-6D10-426F-A426-DDE3E24E4334,Entitlement application,Main,-18008615298673397,mail valid,none,null,097D1E41-3CDB-4652-ABF1-EAEFC0410FA0,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,fb5fa8,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,initialize,none,-72051858488795157,96CBE6E6-9774-4DF8-842B-073F4FDCE2B8,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Sampling the dataframe based on the case id.

In [14]:
#sample the dataframe according to unique case id-s

selected_names = np.random.choice(original_dataframe["case:concept:name"].unique(),8000,replace = False)
df_sampled = original_dataframe[original_dataframe["case:concept:name"].isin(selected_names)]

#export the sampled dataframe
sampled_dataframe = df_sampled.to_csv('sampled_dataframe.csv', index = True)

In [15]:
df_sampled.head()

,success,org:resource,docid_uuid,doctype,subprocess,docid,activity,note,eventid,identity:id,...,case:concept:name,case:amount_applied1,case:payment_actual1,case:penalty_amount1,case:amount_applied2,case:payment_actual2,case:penalty_amount2,case:amount_applied3,case:payment_actual3,case:penalty_amount3
4,True,fb5fa8,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,initialize,none,-72051858488795157,96CBE6E6-9774-4DF8-842B-073F4FDCE2B8,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,True,dde669,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,begin editing,none,-72051856766451566,9EED21E6-02AD-46B6-8A9A-8444F6A7D023,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,True,dde669,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,save,none,-72051856766451552,76920445-7689-4283-A471-9BAA9BC63F72,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,True,dde669,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,insert document,none,-72051856766451513,58C50CA0-EC73-4A86-93E0-80405ED5B0C5,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,True,dde669,CCBAA174-CDD7-4D32-892E-F14197C65B8A,Parcel document,Main,-72051858488795160,save,none,-72051856766451254,EC68AF3B-3C3E-4138-B0E6-3ABBEF3A2C92,...,8b99873a6136cfa6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Sorting the sampled dataframe by timestamp.

In [16]:
pm4py_dataframe = pm4py.format_dataframe(df_sampled, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp')
dataframe = pm4py.convert_to_dataframe(pm4py_dataframe)
dataframe = dataframe.sort_values(['time:timestamp'], ascending = True, ignore_index = True)

Grouping acvities according to the case id and sorting by timestamp within each case.

In [17]:
df_group_case = dataframe.sort_values(['case:concept:name','time:timestamp'],ascending=True,ignore_index = True).groupby('case:concept:name')
df_ungrouped_ordered = df_group_case.apply(lambda x: x)

df_group_case.head()

,success,org:resource,docid_uuid,doctype,subprocess,docid,activity,note,eventid,identity:id,...,case:payment_actual1,case:penalty_amount1,case:amount_applied2,case:payment_actual2,case:penalty_amount2,case:amount_applied3,case:payment_actual3,case:penalty_amount3,case:concept:name,@@index
0,True,0;n/a,00004222-2A14-4021-9AB1-2142397DE7B6,Payment application,Application,-54037160734992421,mail income,none,null,8408B373-0D7D-43CB-95A3-F485DD44A4BD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00004222-2A14-4021-9AB1-2142397DE7B6,0
1,True,0;n/a,00004222-2A14-4021-9AB1-2142397DE7B6,Payment application,Application,-54037160734992421,mail valid,none,null,C99C4A29-47FB-43A6-A9F5-4FCC678B556C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00004222-2A14-4021-9AB1-2142397DE7B6,1
2,True,Document processing automaton,00004222-2A14-4021-9AB1-2142397DE7B6,Payment application,Application,-54037160734992421,initialize,none,-54037160734992422,1B5C1F7D-B9ED-43C1-B2DB-E03497AE2A4F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00004222-2A14-4021-9AB1-2142397DE7B6,2
3,True,727350,00004222-2A14-4021-9AB1-2142397DE7B6,Payment application,Application,-54037160734992421,begin editing,automatic,-18008355034696241,4C81F8FB-EE8C-4F6C-8BDA-D2ADA92BD0F3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00004222-2A14-4021-9AB1-2142397DE7B6,3
4,True,727350,00004222-2A14-4021-9AB1-2142397DE7B6,Payment application,Application,-54037160734992421,calculate,automatic,-18008355034689363,EA7C4FDA-67FB-4EA4-8EBC-D937BD53CB8F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00004222-2A14-4021-9AB1-2142397DE7B6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174829,True,Document processing automaton,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,Geo parcel document,Main,-18008418265078828,initialize,none,-18008418265078826,C2DE7F41-ED07-4E8F-B60F-236A2B0B3CC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,174829
174830,True,Document processing automaton,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,Geo parcel document,Declared,-18008418265078828,begin editing,none,-18008418265078737,EE51E19C-07E8-4D3C-BB81-BBE58E30B3AD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,174830
174831,True,Document processing automaton,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,Geo parcel document,Declared,-18008418265078828,finish pre-check,none,-18008418265078729,DF23DA2D-234D-40A5-A1FF-81B6AAE2499C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,174831
174832,True,Document processing automaton,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,Geo parcel document,Declared,-18008418265078828,finish editing,none,-18008418265078590,662FDCE6-1C26-4640-84AD-761A05B6E696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FFF8BE67-8FC1-4C80-BB5B-1FE439A93283,174832


1.  Calculating the difference between the first event in the prefix (the start time of the case) and the first event in the log (the start time of the log’s timeframe).
2.  Encoding the cases in which the Treatment Activity occurs.

In [30]:
treatment_activity = 'revoke decision'

time_first_activity = dataframe['time:timestamp'].iloc[0]
dt_first = time_first_activity.strftime('%Y-%m-%d %H:%M:%S')
dt_first = datetime.strptime(dt_first, '%Y-%m-%d %H:%M:%S')

df_1 = pd.DataFrame(columns = ['Case Id','Time_Since_First_Activity','Treatment'])
case_count = 0
count_activity = 0

for name, group in tqdm(df_group_case):
    new_row = {'Case Id': df_ungrouped_ordered.iloc[count_activity]['case:concept:name'], 'Time_Since_First_Activity': -1, 'Treatment': 0}
    start_activity = df_ungrouped_ordered.iloc[count_activity]['concept:name']
    df_1 = df_1.append(new_row,ignore_index=True)
    
    #iterating through the activities of each case
    for val in group["concept:name"]:
        
        #calculating time since first activity of the log for each case
        if val == start_activity:
            dt_str_b = df_ungrouped_ordered.iloc[count_activity]['time:timestamp']
            dt_b = dt_str_b.strftime('%Y-%m-%d %H:%M:%S')
            dt_b = datetime.strptime(dt_b, '%Y-%m-%d %H:%M:%S')
            df_1['Time_Since_First_Activity'].iloc[case_count] = (dt_b - dt_first).total_seconds()
        
        #encoding the Treatment variable as 1 if the Treatment Activity is executed in the current case
        if val == treatment_activity:                   
            df_1['Treatment'].iloc[case_count] = 1
            
        count_activity += 1
    case_count+=1


100%|██████████| 16154/16154 [05:08<00:00, 52.34it/s]


In [31]:
df_1.head()

,Case Id,Time_Since_First_Activity,Treatment
0,00103CCB-8B0D-45BF-AE47-FB13C38B8A02,70664597.0,0
1,001070C5-D36C-4C9C-82F7-FECE74B4DBA2,49049230.0,0
2,0010C03B-78B9-440D-98C2-08D0BD4FFC40,73700819.0,0
3,001120A7-5D3B-4FAA-BC45-7437E8EADE0F,8000508.0,0
4,0011DE23-A247-40D1-A68A-556A8A0359AF,49049591.0,0


Encoding the prefix for each case.  In cases where the treatment assignment is present in the data, the last activity of the prefix is the activity occuring before the treatment activity. In cases where the treatment did not occur, the treatment point is estimated by randomly drawing from the distribution of prefix lengths when the treatment is present.

In [18]:
treatment_activity = 'revoke decision'
count_activity = 0
proxy = []

for name, group in tqdm(df_group_case):
    count_activity = 0
    
    #iterating through the activities of each case
    for val in group["concept:name"]:
        
        #counting the activities before treatment
        if val == treatment_activity:  
            proxy.append(count_activity)
            break
        count_activity += 1
        
min_proxy = min(proxy)
max_proxy = max(proxy)


100%|██████████| 16000/16000 [00:02<00:00, 5689.08it/s]


In [ ]:
df_2 = pd.DataFrame()
count_activity_l = 0
df_3 = pd.DataFrame(columns = ['Case Id','Start Activity Time','End Activity Time','Prefix Time'])

for name, group in tqdm(df_group_case):
    flag = False
    count_activity_c = 0
    ct_treatment = 0
    #iterating through the activities of each case
    for val in group["concept:name"]:
        # checking if the case has a treatment
        if val == treatment_activity:
            #checking if the treatment occurs for the first time or not inside the case
            if flag == True:
                ct_treatment = ct_treatment
            else:
                ct_treatment = count_activity_c
                flag = True
                
        if val == start_activity:
            start_activity_time = df_ungrouped_ordered.iloc[count_activity_l]['time:timestamp']
            
        if val == end_activity:
            end_activity_time = df_ungrouped_ordered.iloc[count_activity_l]['time:timestamp']
        
        count_activity_l +=1
        count_activity_c +=1   
          
    #in the cases that have a treatment, insert in the dataframe n activities that occur before the treatment
    if flag == True:
        i=0
        for x in range(1,ct_treatment+1):
            df_2 = df_2.append(df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i])
            i-=1
            
        new_row = {'Case Id': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['case:concept:name'],'Start Activity Time':start_activity_time,
                   'End Activity Time': end_activity_time, 'Prefix Time': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['time:timestamp']}
        df_3 = df_3.append(new_row,ignore_index=True)  
        
    #in the cases that don't have a treatment, insert in the dataframe the first n activities 
    elif flag == False:
        i=0
        j = random.randint(min_proxy,max_proxy)
        
        #check if the randomly generated number is larger than the number of activitites in the cases that don't have a treatment
        if count_activity_c < j:
            for x in range(1,count_activity_c+1):
                df_2 = df_2.append(df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i])
                i-=1
                
            new_row = {'Case Id': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['case:concept:name'],'Start Activity Time': start_activity_time,
                       'End Activity Time': end_activity_time, 'Prefix Time': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['time:timestamp']}
            df_3 = df_3.append(new_row,ignore_index=True)  
        
        else:
            for x in range(1,j+1):
                df_2 = df_2.append(df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i])
                i-=1
                
            new_row = {'Case Id': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['case:concept:name'],'Start Activity Time':start_activity_time,
                       'End Activity Time': end_activity_time, 'Prefix Time': df_ungrouped_ordered.iloc[count_activity_l-count_activity_c-i-1]['time:timestamp']}
            df_3 = df_3.append(new_row,ignore_index=True)  
            
df_2 = df_2.reset_index()
df_3 = df_3.reset_index()


 22%|██▏       | 3469/16000 [25:13<2:22:39,  1.46it/s]

In [ ]:
df_2.head()

In [ ]:
df_3.head()

Use aggregation encoding for the Activity and Resource columns.

In [ ]:
df_4 = pd.crosstab(df_2['case:concept:name'],df_2['concept:name'],colnames = [None]).reset_index()
df_5 = pd.crosstab(df_2['case:concept:name'],df_2['org:resource'],colnames = [None]).reset_index()

In [ ]:
df_4.head()

In [ ]:
df_5.head()

1.  Calculate the duration from the start activity until the end activity of the prefix of each case.
2.  Calculate the month, weekday, hour of the last event in the prefix for each case.
3.  Calculate the open cases when the last activity in the prefix occurs.

In [ ]:
df_6 = pd.DataFrame(columns = ['Case Id','Duration','Month','Weekday','Hour','Open Cases'])

for index, row in df_3.iterrows():
    dt_start = row['Start Activity Time']
    dt_end = row['End Activity Time']
    dt_prefix_time = row['Prefix Time']
    
    dt_start = dt_start.strftime('%Y-%m-%d %H:%M:%S')
    dt_start = datetime.strptime(dt_start, '%Y-%m-%d %H:%M:%S')
    dt_end = dt_end.strftime('%Y-%m-%d %H:%M:%S')
    dt_end = datetime.strptime(dt_end, '%Y-%m-%d %H:%M:%S')
    dt_prefix_string = dt_prefix_time.strftime('%Y-%m-%d %H:%M:%S')
    dt_prefix = datetime.strptime(dt_prefix_string, '%Y-%m-%d %H:%M:%S')
    
     
    weekday = dt_prefix_time.weekday()
    dt_prefix_string = dt_prefix_string.replace("-", " ") 
    dt_prefix_string = dt_prefix_string.replace(":"," ")
    input_tokens = word_tokenize(dt_prefix_string)
    month = input_tokens[1]
    hour = input_tokens[3]
    
    open_cases = 0
    
    for index2, row2 in df_3.iterrows():
        dt_start2 = row2['Start Activity Time']
        dt_end2 = row2['End Activity Time']

        dt_start2 = dt_start2.strftime('%Y-%m-%d %H:%M:%S')
        dt_start2 = datetime.strptime(dt_start2, '%Y-%m-%d %H:%M:%S')
        dt_end2 = dt_end2.strftime('%Y-%m-%d %H:%M:%S')
        dt_end2 = datetime.strptime(dt_end2, '%Y-%m-%d %H:%M:%S')
        
        if (dt_start2 - dt_prefix).total_seconds() < 0 and (dt_end2 - dt_prefix).total_seconds() > 0:
            open_cases += 1
        
    new_row = {'Case Id': row['Case Id'],'Duration': (dt_prefix - dt_start).total_seconds() ,
               'Month': month, 'Weekday': weekday, 'Hour': hour, 'Open Cases': open_cases}
    df_6 = df_6.append(new_row,ignore_index=True)

Outputting the created dataframes as csv files.

In [ ]:
df1 = df_1.to_csv('df_1.csv', index = True)
df2 = df_2.to_csv('df_2.csv', index = True)
df3 = df_3.to_csv('df_3.csv', index = True)
df4 = df_4.to_csv('df_4.csv', index = True)
df5 = df_5.to_csv('df_5.csv', index = True)
df6 = df_6.to_csv('df_6.csv', index = True)